In [1]:
import os
import sys
import gzip
import threading
import argparse
import pandas as pd
import numpy as np
from Bio import SeqIO

In [2]:
def ReadFile(filename):
    if filename.endswith('.gz'):
        f = gzip.open(filename, 'rt')
    else:
        f = open(filename)
    return(f)


def WriteFile(filename):
    if filename.endswith('.gz'):
        f = gzip.open(filename, 'wt')
    else:
        f = open(filename, 'wt')
    return(f)


def UniProtFastaParse(record):
    '''
    [UniProtKB]
    >db|UniqueIdentifier|EntryName ProteinName OS=OrganismName OX=OrganismIdentifier [GN=GeneName ]PE=ProteinExistence SV=SequenceVersion

    SeqRecord(seq=Seq('MEEMYNVSSNPHVRDKMTTSRIMQLVVIALLPATLFGIWNFGFHALLVVLVTVI...KKA'),
        id='tr|A0A173W9M9|A0A173W9M9_9FIRM',
        name='tr|A0A173W9M9|A0A173W9M9_9FIRM',
        description='tr|A0A173W9M9|A0A173W9M9_9FIRM Ion-translocating oxidoreductasecomplex subunit D OS=Blautia obeum OX=40520 GN=nqrB PE=3 SV=1',
        dbxrefs=[])
    '''
    name = record.name
    db, UniqueIdentifier, EntryName = name.split("|")
    seq = str(record.seq)
    desc = record.description
    desc = desc.replace("{} ".format(name), "", 1)
    # print(desc)
    return(db, UniqueIdentifier, EntryName, desc, seq)


def UniProtFasta2tsv(UniprotDB_file, df_Uniprot_list, index, columns):
    in_f = ReadFile(UniprotDB_file)
    tmp = []
    for record in SeqIO.parse(in_f, 'fasta'):
        new_row = UniProtFastaParse(record)
        tmp.append(new_row)
    in_f.close()
    df_Uniprot_list[index] = pd.DataFrame(tmp, columns=columns)

def MQfileParse(filepath, PEP_threshold, df_Uniprot):
    import re
    '''
    Within the target-decoy strategy MaxQuant use the concept of posterior error probability (PEP) 
    to integrate multiple peptide properties, such as length, charge and number of modifications, 
    together with the Andromeda score into a single quantity reflecting the quality of a peptide 
    spectrum match (PSM)
    '''
    def Filterresult(df, PEP_threshold):
        return(df.loc[(df['Reverse'] != '+') & (df['Potential contaminant'] != '+') & (df['PEP'] < PEP_threshold) & (df['Proteins'] != 'Biognosys')])

    def Proteins2Species(df, df_Uniprot):
        if 'Proteins' in df.columns:
            # df['Species'] = df.apply(lambda x: ';'.join([str(i) for i in df_Uniprot.loc[df_Uniprot.UniqueIdentifier.isin(
            #     str(x['Proteins']).split(';')), 'OrganismName'].values]), axis=1)
            df['Species'] = df.apply(lambda x: ';'.join([''.join(df_Uniprot.loc[df_Uniprot.UniqueIdentifier == i, 'OrganismName'].values)
                                                         for i in str(x['Proteins']).split(';')]),axis=1)
            df['Protein number'] = df.apply(lambda x: len(
                set([i for i in str(x['Proteins']).split(';') if i != ""])), axis=1)
            df['Species number'] = df.apply(lambda x: len(
                set([ i for i in str(x['Species']).split(';') if i != ""])), axis=1)
        if 'Leading proteins' in df.columns:
            # df['Leading species'] = df.apply(lambda x: ';'.join([str(i) for i in df_Uniprot.loc[df_Uniprot.UniqueIdentifier.isin(
            #     str(x['Leading proteins']).split(';')), 'OrganismName'].values]), axis=1)
            df['Leading species'] = df.apply(lambda x: ';'.join([''.join(df_Uniprot.loc[df_Uniprot.UniqueIdentifier == i, 'OrganismName'].values)
                                                         for i in str(x['Leading proteins']).split(';')]),axis=1)
            df['Leading proteins number'] = df.apply(lambda x: len(
                set([ i for i in str(x['Leading proteins']).split(';') if i != ""])), axis=1)
            df['Leading species number'] = df.apply(lambda x: len(
                set([ i for i in str(x['Leading species']).split(';') if i != ""])), axis=1)
        if 'Leading razor protein' in df.columns:
            # df['Leading razor species'] = df.apply(lambda x: ';'.join([str(i) for i in df_Uniprot.loc[df_Uniprot.UniqueIdentifier.isin(
            #     str(x['Leading razor protein']).split(';')), 'OrganismName'].values]), axis=1)
            df['Leading razor species'] = df.apply(lambda x: ';'.join([''.join(df_Uniprot.loc[df_Uniprot.UniqueIdentifier == i, 'OrganismName'].values) 
                                                                 for i in str(x['Leading razor protein']).split(';')]),axis=1)
            df['Leading razor protein number'] = df.apply(lambda x: len(
                set([ i for i in str(x['Leading razor protein']).split(';') if i != ""])), axis=1)
            df['Leading razor species number'] = df.apply(lambda x: len(
                set([ i for i in str(x['Leading razor species']).split(';') if i != ""])), axis=1)
    def ProteinRematch(df, df_Uniprot):
        my_sequence = df['Sequence'].str.replace('I','L')
        df_searching = df_Uniprot[['UniqueIdentifier','seq']].copy()
        df_searching['seq'] = df_searching['seq'].str.replace('I','L')
        return(my_sequence.apply(lambda x: ';'.join(df_searching.loc[df_searching['seq'].str.contains(x), 'UniqueIdentifier'].to_list())))
    df_peptides = pd.read_table(filepath)
    df_peptides = Filterresult(df_peptides, PEP_threshold).copy()
    if 'Proteins' in df_peptides.columns:
        df_peptides['Proteins'] = ProteinRematch(df_peptides, df_Uniprot)
    Proteins2Species(df_peptides, df_Uniprot)
    # df_micro_ori_peptides = df_peptides.loc[~df_peptides['Species'].apply(
    #     lambda x: ';'.join((set(x.split(';'))))).isin(['Homo sapiens', '']), ].copy()
    df_micro_ori_peptides = df_peptides.loc[df_peptides['Species'].apply(
        lambda x: len([ i for i in x.split(';') if i not in ['Homo sapiens', '']]) > 0), ].copy()
    return(df_peptides, df_micro_ori_peptides)

def Juge(df):
    human_ori_juge = df['Species'].apply(lambda x: ';'.join((set(x.split(';'))))).isin(['Homo sapiens', '']).sum()
    nonhuman_ori_juge = df['Species'].apply(lambda x: 'Homo sapiens' not in x).sum()
    multi_ori_juge = (df['Species number'] > 1).sum()
    nonhuman_multi_ori_juge = ((df['Species number'] > 1) & (df['Species'].apply(lambda x: 'Homo sapiens' not in x))).sum()
    return(human_ori_juge, nonhuman_ori_juge, multi_ori_juge, nonhuman_multi_ori_juge)

def RunCmd(cmd):
    # print(cmd)
    os.system(cmd)

In [3]:
# netMHCPan Predicting
def AffinityPredicting(df_peptides, prefix, HLAinfo, outdir):
    netMHCPan = "/home/guanxiangyu/netMHCpan-4.1/netMHCpan"
    os.makedirs(outdir, exist_ok=True)
    
    # HLA type included in netMHCPan
    netMHCPan_included_HLA = list()
    tmp = os.popen("{} -listMHC".format(netMHCPan))
    for line in tmp:
        line = line.strip()
        if line and (not line.startswith('#')):
            netMHCPan_included_HLA.append(line)
            
    # HLA type filtering
    if type(HLAinfo) == str and os.path.exists(HLAinfo):
        # for file input
        hla_f = ReadFile(HLAinfo)
        hla_type = hla_f.read().strip().split("\n")
        hla_f.close()
    else:
        # for HLA type list or string
        hla_type = HLAinfo
    hla_type = sorted(set(hla_type))
    for hla in hla_type:
        if hla in netMHCPan_included_HLA:
            continue
        else:
            print(">>>either {} cannot be predicted by netMHCpan-4.1 or your input format could be wrong.<<<".format(hla))
            hla_type.remove(hla)
    
    # netMHCPan predicting
    pep_file = os.path.join(outdir, '{}.pep'.format(prefix))
    df_peptides.Sequence.drop_duplicates().to_csv(pep_file, header=False, index=False, sep="\t")
    cmd = "{} -p {} -BA -inptype 1 -a {} > {}.txt".format(netMHCPan, pep_file, ','.join(hla_type), pep_file)
    RunCmd(cmd)
    cmd = "rm {}".format(pep_file)
    RunCmd(cmd)
    # netMHCPan result processing
    netmhc_result_trans = "/mnt/hgfs/G/Micro_Maxquant_v2/scripts/formattingNetMHCpanResults-n.pl"
    cmd = "{} {}.txt {} {}".format(netmhc_result_trans, pep_file, outdir, prefix)
    RunCmd(cmd) 

In [4]:
os.getcwd()

'/mnt/hgfs/G/Micro_Maxquant_v2/scripts'

In [5]:
file_path = "/mnt/hgfs/G/Micro_Maxquant_v2"
sample_ids = ['CRC01', 'CRC02', 'CRC03', 'CRC04', 'CRC05', 'CRC06', 'CRC07', 'CRC08', 'CRC09', 'CRC10']
dtypes = ['tumor','normal']

In [6]:
PEP_threshold = 0.05
MQ_file = "peptides.txt"
pep_ori_summary = WriteFile(os.path.join(file_path, 'results', 'All.peptides_summarise.tsv.gz'))
pep_ori_summary.write("\t".join(['Sample', 'dtype', 'peptides sequences filtered', 'human origin peptides number', 'non-human origin peptides number',
                                 'multi-origin peptides number', 'non-human multi-origin peptides number']))
pep_ori_summary.write("\n")
OrganismCount = list()
TotalPeptides = list()
TotalNonHumanPeptides = list()
for sample in sample_ids:
    # Uniprot fasta databases parse to tsv format
    columns = ['db', 'UniqueIdentifier', 'EntryName', 'Description', 'seq']
    ref_dfs = list()
    my_threads = list()
    reference = [os.path.join(file_path,"ProteinDB",i) for i in os.listdir(os.path.join(file_path,"ProteinDB")) if i.startswith(sample)]
    for i, ref_fasta in enumerate(reference):
        ref_dfs.append(pd.DataFrame(columns=columns))
        my_threads.append(threading.Thread(target=UniProtFasta2tsv, args=(ref_fasta, ref_dfs, i, columns)))

    for th in my_threads:
        th.start()
    for th in my_threads:
        th.join()
    # combine UniProt human and microbiome databases as one
    df_Uniprot = pd.concat(ref_dfs, ignore_index=True)
    df_Uniprot = pd.concat([df_Uniprot, df_Uniprot.Description.str.extract("OS=(?P<OrganismName>.+)\sOX=(?P<OrganismIdentifier>\d+)", expand=True)], axis=1)
    assert (len(df_Uniprot.UniqueIdentifier.unique()) == df_Uniprot.shape[0]), "There are multi_match in protein database"
    # save as tsv file and count protein numbers in different species
    df_Uniprot.to_csv(os.path.join(file_path, 'results', '{}.MQdb.tsv.gz'.format(sample)), sep="\t", index=False, header=True)
    # df_Uniprot.OrganismName.value_counts().reset_index().to_csv(os.path.join(file_path, 'results', '{}.MQdb_Species_cout.tsv.gz'.format(sample)), sep="\t", index=False, header=True)
    df_Organism = df_Uniprot.OrganismName.value_counts().reset_index()
    df_Organism['Sample'] = sample
    OrganismCount.append(df_Organism)
    # MaxQuant peptides.txt file parse, annotate originated species of each peptides
    for Dtype in dtypes:
        peptide_f = os.path.join(file_path, sample, Dtype, MQ_file)
        HLAinfo = os.path.join(file_path, sample, 'HLAdect.normal.csv')
        if os.path.exists(peptide_f):
            df_peptides, df_Micro_peptides = MQfileParse(peptide_f, PEP_threshold, df_Uniprot)
            sample_outdir = os.path.join(file_path, 'results', sample)
            os.makedirs(sample_outdir, exist_ok=True)
            if df_peptides.shape[0] >0:
                prefix = '{}_{}.all_peptides'.format(sample, Dtype)
                AffinityPredicting(df_peptides, prefix, HLAinfo, sample_outdir)
                df_peptides.to_csv(os.path.join(sample_outdir, '{}.tsv.gz'.format(prefix)), header=True, index=False, sep="\t")          
                df_peptides['Sample'] = sample
                df_peptides['Sample_Type'] = Dtype
                TotalPeptides.append(df_peptides)
            if df_Micro_peptides.shape[0] > 0:
                prefix = '{}_{}.micro_ori_peptides'.format(sample, Dtype)
                AffinityPredicting(df_peptides, prefix, HLAinfo, sample_outdir)
                df_Micro_peptides.to_csv(os.path.join(sample_outdir, '{}.tsv.gz'.format(prefix)), header=True, index=False, sep="\t")
                df_Micro_peptides['Sample'] = sample
                df_Micro_peptides['Sample_Type'] = Dtype
                TotalNonHumanPeptides.append(df_Micro_peptides)
            human_ori_juge, nonhuman_ori_juge, multi_ori_juge, nonhuman_multi_ori_juge = Juge(df_peptides)
            pep_ori_summary.write("{}\t{}\t{:d}\t{:d}\t{:d}\t{:d}\t{:d}\n".format(sample, Dtype, df_peptides.shape[0], human_ori_juge, nonhuman_ori_juge, multi_ori_juge, nonhuman_multi_ori_juge))

pep_ori_summary.close()
df_OrgCount = pd.concat(OrganismCount)
df_OrgCount.to_csv(os.path.join(file_path, 'results', 'All.MQdb_Species_count.tsv.gz'), sep="\t", index=False, header=True)
if len(TotalPeptides) > 0:
    df_PepTotal = pd.concat(TotalPeptides)
    df_PepTotal.to_csv(os.path.join(file_path, 'results', 'All.Peptides.tsv.gz'), header=True, index=False, sep="\t")
if len(TotalNonHumanPeptides) > 0:
    df_MicroPepTotal = pd.concat(TotalNonHumanPeptides)
    df_MicroPepTotal.to_csv(os.path.join(file_path, 'results', 'All.NonHuman.Peptides.tsv.gz'), header=True, index=False, sep="\t")

In [7]:
df_MicroPepTotal.columns

Index(['Sequence', 'N-term cleavage window', 'C-term cleavage window',
       'Amino acid before', 'First amino acid', 'Second amino acid',
       'Second last amino acid', 'Last amino acid', 'Amino acid after',
       'A Count', 'R Count', 'N Count', 'D Count', 'C Count', 'Q Count',
       'E Count', 'G Count', 'H Count', 'I Count', 'L Count', 'K Count',
       'M Count', 'F Count', 'P Count', 'S Count', 'T Count', 'W Count',
       'Y Count', 'V Count', 'U Count', 'O Count', 'Length',
       'Missed cleavages', 'Mass', 'Proteins', 'Leading razor protein',
       'Start position', 'End position', 'Gene names', 'Protein names',
       'Unique (Groups)', 'Unique (Proteins)', 'Charges', 'PEP', 'Score',
       'Intensity', 'Reverse', 'Potential contaminant', 'id',
       'Protein group IDs', 'Mod. peptide IDs', 'Evidence IDs', 'MS/MS IDs',
       'Best MS/MS', 'Deamidation (NQ) site IDs', 'Oxidation (M) site IDs',
       'Taxonomy IDs', 'Mass deficit', 'MS/MS Count', 'Species',
       'Pr

In [8]:
df_MicroPepTotal[['Proteins','Species']][df_MicroPepTotal.Proteins.str.contains(';')]

,Proteins,Species
311,Q12931;Q12931-2;P14625;P08238;A0A2U8FNC1;A0A0S...,Homo sapiens;Homo sapiens;Homo sapiens;Homo sa...
626,Q14568;P14625;P08238;P07900;P07900-2;A0A2U8FNC...,Homo sapiens;Homo sapiens;Homo sapiens;Homo sa...
1192,P23526;P23526-2;A0A2U8FVX6,Homo sapiens;Homo sapiens;Aquabacterium olei
680,P05023;P05023-3;P05023-4;P50993;P20648;Q13733;...,Homo sapiens;Homo sapiens;Homo sapiens;Homo sa...
909,Q14568;P14625;P08238;P07900;P07900-2;A0A2U8FNC...,Homo sapiens;Homo sapiens;Homo sapiens;Homo sa...
...,...,...
758,Q8NE71;Q8NE71-2;A0A0B4RZK9,Homo sapiens;Homo sapiens;Parvimonas micra
1011,P35606;P35606-2;Q8REL3,Homo sapiens;Homo sapiens;Fusobacterium nuclea...
1202,A0A378C574;A0A378AJY0;F9EL03;A6TEX7;A0A378E6C6...,Klebsiella pneumoniae subsp. ozaenae;Klebsiell...
1227,P38919;P60842;P60842-2;Q14240;Q14240-2;A0A378A...,Homo sapiens;Homo sapiens;Homo sapiens;Homo sa...


In [9]:
df_PepTotal.drop_duplicates('Sequence').shape

(14289, 67)

In [10]:
df_MicroPepTotal.drop_duplicates('Sequence').shape

(230, 67)